### XGBoost+LR Model

In [1]:
from sklearn.tree import DecisionTreeClassifier
from lightgbm.sklearn import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn import metrics
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
import lightgbm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import scipy.io as sio

### 1.准备数据

In [2]:
dpath = "./data/"

In [3]:
data_train = pd.read_csv(dpath+"data_train.csv")
data_train.shape

(15398, 10)

In [4]:
data_train.head()

,invited,user_reco,evt_p_reco,evt_c_reco,user_pop,frnd_infl,evt_pop,lfm_reco,interested,not_interested
0,0,0.000000,9.800970e-01,9.800970e-01,0.000231,0.0,-0.000039,0.00548,0,0
1,0,0.000000,1.821861e-01,1.821861e-01,0.000231,0.0,0.000018,-0.12674,0,0
2,0,143.279558,-1.000000e+00,-1.000000e+00,0.000231,0.0,0.000173,0.68928,1,0
3,0,0.000000,1.175019e+00,1.175019e+00,0.000231,0.0,0.000016,0.04961,0,0
4,0,32.328494,2.239399e-07,2.239399e-07,0.000231,0.0,0.000064,0.18320,0,0


In [6]:
X_train = np.array(data_train.iloc[:,:-2])
y_train = np.array(data_train.iloc[:,-2])

### 2. xgboost提取特征

In [7]:
%%time
params = {'learning_rate': 0.05,
          'n_estimators': 20, 
          'max_depth': 5,
          'subsample': 0.6,
          'colsample_bytree':0.5,
          'nthread': 4,
         }
xgb = XGBClassifier(**params)
xgb.fit(X_train,y_train)

Wall time: 89.9 ms


In [8]:
#保存模型
pickle.dump(xgb,open("./model/xgb.pkl","wb"))

In [9]:
%%time
X_train_leaf = xgb.apply(X_train)
X_train_leaf.shape

Wall time: 30 ms


In [ ]:
#保存X_train_leaf
sio.mmwrite("./data/X_train_leaf", X_train_leaf)

In [ ]:
X_train_leaf = sio.mmread("./data/X_train_leaf")

In [10]:
X_train_leaf.shape

(15398, 20)

In [11]:
%%time
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()
X_train_enc = enc.fit_transform(X_train_leaf)

Wall time: 60 ms


D:\Program Files\Anaconda\lib\site-packages\sklearn\preprocessing\_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [12]:
X_train_enc.shape

(15398, 408)

In [16]:
xgb.feature_importances_

array([0.01030928, 0.0927835 , 0.38659793, 0.13402061, 0.09020618,
       0.        , 0.1185567 , 0.16752577], dtype=float32)

### 3.训练LR

In [13]:
%%time
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(n_jobs=4, penalty='l1')
lr.fit(X_train_enc,y_train)

Wall time: 90 ms


D:\Program Files\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
D:\Program Files\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:1300: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 4.
  " = {}.".format(effective_n_jobs(self.n_jobs)))


In [14]:
y_proba = lr.predict_proba(X_train_enc)
auc = roc_auc_score(y_train,y_proba[:,1])
print(auc)

0.9998852483104312


In [15]:
#保存预测结果
pickle.dump(y_proba,open("./result/xgb_lr_pred.pkl","wb"))